<a href="https://colab.research.google.com/github/GKurek94/Data-Science-projects/blob/main/Titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Titanic competition solution transfered to Google Collaboratory.

## Data Preprocessing 

Installing required library

In [3]:
!pip install catboost


     |████████████████████████████████| 76.3 MB 24 kB/s 


Importing the required libraries. Data load. First moment of preprocessing which is changing empty values to 0 or average.



In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split




train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df['Cabin'] = train_df['Cabin'].fillna(0)
test_df['Cabin'] = test_df['Cabin'].fillna(0)
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
print(train_df.isnull().sum())
train_df.duplicated().sum()
#train_target = train_df.Survived
#train_data = train_df.drop('Survived', axis=1)
#test_df.head()



PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64


0

In [4]:
from catboost import CatBoostClassifier

Splitting the data. 

In [26]:

from xgboost import XGBClassifier

In [5]:
train_target = train_df.Survived
train_data = train_df.drop('Survived', axis=1)

For ease of project dropping column containing the name of passengers. Creating Title column with most common titles of people on Titanic.

In [8]:
train_data['Title'] = np.where(train_data['Name'].str.find('Mrs') != -1, 'Mrs.', train_data['Name'])
train_data['Title'] = np.where(train_data['Name'].str.find('Miss.') != -1, 'Miss', train_data['Title'])
train_data['Title'] = np.where(train_data['Name'].str.find('Mr.') != -1, 'Mr.', train_data['Title'])
train_data['Title'] = np.where(train_data['Name'].str.find('Master') != -1, 'Master', train_data['Title'])
train_data['Title'] = np.where(train_data['Name'].str.find('Dr.') != -1, 'Dr.', train_data['Title'])
train_data['Title'] = np.where(train_data['Name'].str.find('Rev.') != -1, 'Rev.', train_data['Title'])
train_data['Title'] = np.where((train_data['Name'].str.find('Rev.') == -1) & (train_data['Name'].str.find('Dr.') == -1) &
                               (train_data['Name'].str.find('Master.') == -1) & (train_data['Name'].str.find('Mr.') == -1) &
                               (train_data['Name'].str.find('Mrs.') == -1) & (train_data['Name'].str.find('Miss.') == -1), 
                               'Other_title.', train_data['Title'])
train_data = train_data.drop('Name', axis=1)

KeyError: ignored

Same thing happens to test database.

In [11]:
test_df['Title'] = np.where(test_df['Name'].str.find('Mrs') != -1, 'Mrs.', test_df['Name'])
test_df['Title'] = np.where(test_df['Name'].str.find('Miss.') != -1, 'Miss', test_df['Title'])
test_df['Title'] = np.where(test_df['Name'].str.find('Mr.') != -1, 'Mr.', test_df['Title'])
test_df['Title'] = np.where(test_df['Name'].str.find('Master') != -1, 'Master', test_df['Title'])
test_df['Title'] = np.where(test_df['Name'].str.find('Dr.') != -1, 'Dr.', test_df['Title'])
test_df['Title'] = np.where(test_df['Name'].str.find('Rev.') != -1, 'Rev.', test_df['Title'])
test_df['Title'] = np.where((test_df['Name'].str.find('Rev.') == -1) & (test_df['Name'].str.find('Dr.') == -1) &
                               (test_df['Name'].str.find('Master.') == -1) & (test_df['Name'].str.find('Mr.') == -1) &
                               (test_df['Name'].str.find('Mrs.') == -1) & (test_df['Name'].str.find('Miss.') == -1), 
                               'Other_title.', test_df['Title'])
test_df = test_df.drop('Name', axis=1)

KeyError: ignored

In [12]:
train_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,3,male,22.000000,1,0,A/5 21171,7.2500,0,S,Mr.
1,2,1,female,38.000000,1,0,PC 17599,71.2833,C85,C,Mrs.
2,3,3,female,26.000000,0,0,STON/O2. 3101282,7.9250,0,S,Miss
3,4,1,female,35.000000,1,0,113803,53.1000,C123,S,Mrs.
4,5,3,male,35.000000,0,0,373450,8.0500,0,S,Mr.
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,male,27.000000,0,0,211536,13.0000,0,S,Rev.
887,888,1,female,19.000000,0,0,112053,30.0000,B42,S,Miss
888,889,3,female,29.699118,1,2,W./C. 6607,23.4500,0,S,Miss
889,890,1,male,26.000000,0,0,111369,30.0000,C148,C,Mr.


In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Ticket       891 non-null    object 
 7   Fare         891 non-null    float64
 8   Cabin        891 non-null    object 
 9   Embarked     889 non-null    object 
 10  Title        891 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


Another operation. There are around 150 cabins for survivors. For making calculations easier every cabin is equal to 1, which makes only 2 different values.

In [14]:
train_data['Cabin'] = np.where(train_data['Cabin'] != 0, 1, 
                               train_data['Cabin'])
test_df['Cabin'] = np.where(test_df['Cabin'] != 0, 1, 
                               test_df['Cabin'])

In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Ticket       891 non-null    object 
 7   Fare         891 non-null    float64
 8   Cabin        891 non-null    object 
 9   Embarked     889 non-null    object 
 10  Title        891 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [16]:
train_data['Ticket'].value_counts()

1601                7
347082              7
CA. 2343            7
347088              6
3101295             6
                   ..
SOTON/OQ 3101316    1
250648              1
A/5. 13032          1
350043              1
347083              1
Name: Ticket, Length: 681, dtype: int64

Deleting ticket column, which shouldn't have impact into predicting survival.

In [17]:
train_data = train_data.drop('Ticket', axis=1)
test_df = test_df.drop('Ticket', axis=1)

In [18]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Cabin        891 non-null    object 
 8   Embarked     889 non-null    object 
 9   Title        891 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


Changing type of cabin column to numeric.

In [19]:
train_data['Cabin'] = train_data['Cabin'].astype('int')
test_df['Cabin'] = test_df['Cabin'].astype('int')

In [20]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Cabin        891 non-null    int64  
 8   Embarked     889 non-null    object 
 9   Title        891 non-null    object 
dtypes: float64(2), int64(5), object(3)
memory usage: 69.7+ KB


Convert  variables into dummies.

In [21]:
train_data = pd.get_dummies(train_data)
test_df = pd.get_dummies(test_df)

In [22]:
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].mean())

## Model

Application which contains functions and all argorithms required to find best classifier for titanic survival prediction.

In [ ]:


def best_classifier(dicts):
        for e in dicts:
            if dicts[e] == max(dicts.values()):
                return e


# Logistic Regression
log_reg = LogisticRegression(random_state=0)
log_reg.fit(train_data, train_target)
y_pred_log = log_reg.predict(test_df)
log_score = log_reg.score(train_data, train_target)

# Multinomial Naive Bayes
classifier_multi = MultinomialNB()
classifier_multi.fit(train_data, train_target)
y_pred = classifier_multi.predict(test_df)
multi_score = classifier_multi.score(train_data, train_target)


# Decision Tree Classifier
classifier_tree = DecisionTreeClassifier(max_depth=2, random_state=0)
classifier_tree.fit(train_data, train_target)
y_pred_tree = classifier_tree.predict(test_df)
tree_score = classifier_tree.score(train_data, train_target)

# Linear SVC
classifier_svc = SVC(C=1.0, kernel='linear')
classifier_svc.fit(train_data, train_target)
y_pred_svc = classifier_svc.predict(test_df)
svc_score = classifier_svc.score(train_data, train_target)

# Catboost
cat_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.01, depth=4)
cat_classifier.fit(train_data, train_target)
cat_pred = cat_classifier.predict(test_df)
cat_score = cat_classifier.score(train_data, train_target)

#XGBoost got better score at training 
# xg_model = XGBClassifier()
# model.fit(train_data, train_target)
# xg_pred = model.predict(test_df)
# xgb_score = model.score(train_data, train_target)
# print(xgb_score, cat_score)

# Dictionary of algorithm names and their values in that case
dict_of_algorithms = {
"LogisticRegression": log_score,
"MultinomialNB": multi_score,
"DecisionTreeClassifier": tree_score,
"LinearSVC": svc_score,
"CatBoost": cat_score}
#"XGBoost": xgb_score

results = {
"LogisticRegression": log_score,
"MultinomialNB": multi_score,
"DecisionTreeClassifier": tree_score,
"LinearSVC": svc_score,
"CatBoost": cat_score,
"Best Classifier": best_classifier(dict_of_algorithms),
"Best value": max(log_score, multi_score, tree_score, 
                  svc_score, cat_score)
}
print(results)

In [47]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": cat_pred
    })
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [49]:
submission.to_csv('submission5.csv', index=False)